In [1]:
from scrapping.images import get_images
from scrapping.box import get_box_labels

get_images()

get_box_labels()

In [2]:
import pickle
cosa = "cropped_image_7"
image_path = f"data/cropped/valorant_americas/{cosa}.png"
with open(f"data/box/valorant_americas/{cosa}.pkl", "rb") as box_dictionary_labels:
    labels = pickle.load(box_dictionary_labels)

In [3]:
import numpy as np
import re

In [4]:
def remove_sided_rectangles(labels):
    result = []
    for i in labels:
        top_left = i[0][0]
        top_right = i[0][1]
        bottom_right = i[0][2]
        bottom_left = i[0][3]
        if (top_left[1] == top_right[1]):       
            result.append(i) 
    return result

In [5]:
def fix_horizontal_boxes(labels):    
    horizontal_sequences = []
    current_horizontal_sequence = []
    for i in range(len(labels) - 1):
        top_left = labels[i][0][0]
        top_right = labels[i][0][1]
        bottom_right = labels[i][0][2]
        bottom_left = labels[i][0][3]
    
        top_left_next = labels[i + 1][0][0]
        top_right_next = labels[i + 1][0][1]
        bottom_right_next = labels[i + 1][0][2]
        bottom_left_next = labels[i + 1][0][3]
    
        if (np.abs(bottom_left_next[1] - bottom_right[1]) < 7) & (np.abs(top_left_next[1] - top_right[1]) < 7):
            current_horizontal_sequence.append(i)
        else:
            if len(current_horizontal_sequence) > 0:
                horizontal_sequences.append(current_horizontal_sequence)
            current_horizontal_sequence = []
    [i.append(i[-1] + 1) for i in horizontal_sequences]

    changed_boxes_dict = {}

    for sequence in horizontal_sequences:
        
        text = " "
        y_bottom = -10000 
        y_top = 10000   
        x_left = 10000  
        x_right = -10000 
        
        for index in sequence:    
            top_left = labels[index][0][0]
            top_right = labels[index][0][1]
            bottom_right = labels[index][0][2]
            bottom_left = labels[index][0][3]
    
            if bottom_left[1] > y_bottom:
                y_bottom = bottom_left[1]
                
            if top_left[1] < y_top:
                y_top = top_left[1]
                
            if top_left[0] < x_left:
                x_left = top_left[0]
                
            if top_right[0] > x_right:
                x_right = top_right[0]    
                
            text = text + labels[index][1] + " "
            coordinates = [[x_left,y_top], [x_right,y_top], [x_right,y_bottom], [x_left,y_bottom]]
        
    
        changed_boxes_dict[str(sequence)] = (coordinates, text)   


    summed_text = ""
    for i in changed_boxes_dict.keys():
        summed_text = summed_text + i
    indexes = re.findall(r'\d+', summed_text)
    indexes = [int(index) for index in indexes]

    final_labels = []
    
    for i in range(len(labels)):
        box, text = labels[i][0],labels[i][1]       
        if i not in indexes:
            final_labels.append((box,text))
            
    for v in changed_boxes_dict.values():
        final_labels.append(v)
    

    return final_labels

In [6]:
remove_sided_rectangles_labels = remove_sided_rectangles(labels)
fix_horizontal_boxes_labels = fix_horizontal_boxes(remove_sided_rectangles_labels)

In [7]:
import matplotlib.pyplot as plt

In [8]:
import cv2
import numpy as np

In [9]:
from PIL import Image
import numpy as np

In [10]:

img = cv2.imread(image_path)

# Draw bounding boxes and text
for box, text in fix_horizontal_boxes_labels:
    # Convert box to numpy array and ensure integer type
    pts = np.array(box, np.int32)
    pts = pts.reshape((-1, 1, 2))

    # Draw bounding box
    cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # Add text
    org = (int(box[0][0]), int(box[0][1]) - 5)  # Ensure coordinates are integers
    cv2.putText(img, text, org, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Save the image with bounding boxes and text
cv2.imwrite("image_with_boxes_and_text.jpg", img)

True